In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install datasets --quiet

import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 데이터셋 경로 설정
data_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data/'  # 이미지 및 텍스트 파일 폴더

# 텍스트 전처리 함수
def preprocess_text(text):
    text = text.lower()  # 소문자로 변환
    text = ''.join(e for e in text if e.isalnum() or e.isspace())  # 특수문자 제거
    return text

# 커스텀 데이터셋 클래스 정의
class ImageTextDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.data = []

        # 폴더 내 파일 목록 가져오기
        file_names = [f.split('.')[0] for f in os.listdir(folder_path) if f.endswith('.jpg')]  # .jpg 파일 기준
        unique_ids = set(file_names)  # 중복 제거

        for file_id in unique_ids:
            # 이미지와 텍스트 파일 경로 생성
            image_file = os.path.join(folder_path, f"{file_id}.jpg")
            text_file = os.path.join(folder_path, f"{file_id}.txt")

            # 이미지와 텍스트 파일이 모두 있는 경우 로드
            if os.path.exists(image_file) and os.path.exists(text_file):
                self.data.append((image_file, text_file))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_file, text_file = self.data[idx]

        # 이미지 로드 및 전처리
        image = cv2.imread(image_file)
        if image is None:
            raise ValueError(f"이미지를 로드할 수 없습니다: {image_file}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환

        if self.transform:
            image = self.transform(image)
        else:
            # 기본 변환 적용
            image = cv2.resize(image, (224, 224))
            image = image / 255.0  # 정규화
            image = torch.tensor(image, dtype=torch.float32)
            image = image.permute(2, 0, 1)  # (H, W, C) -> (C, H, W)

        # 텍스트 로드 및 전처리
        with open(text_file, 'r') as file:
            raw_text = file.read().strip()
        text = preprocess_text(raw_text)

        return image, text

# 데이터셋 및 데이터로더 생성
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = ImageTextDataset(data_path, transform=transform)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

## 2. 모델과 라벨 1차 평가

In [11]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from collections import Counter
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, Dataset

# 1. 라벨 파일 로드 및 전처리
label_file_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/label.txt'

# 라벨 파일 로드
label_df = pd.read_csv(label_file_path, sep='\t', header=0)

# 컬럼 이름 지정 (필요에 따라)
label_df.columns = ['ID', 'Annotator1', 'Annotator2', 'Annotator3']

# 감성 라벨 매핑
label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}

# 다수결 라벨 계산 함수
def get_majority_label(labels):
    label_counts = Counter(labels)
    majority_label = label_counts.most_common(1)[0][0]
    return majority_label

# 각 샘플에 대한 다수결 라벨 계산
majority_labels = []
sample_ids = label_df['ID'].tolist()
for idx, row in label_df.iterrows():
    labels = []
    for annotator in ['Annotator1', 'Annotator2', 'Annotator3']:
        text_label = row[annotator].split(',')[0].strip()  # 텍스트 라벨 추출
        labels.append(text_label)
    # 라벨 매핑 적용
    mapped_labels = [label_mapping[label] for label in labels]
    # 다수결 라벨 계산
    majority_label = get_majority_label(mapped_labels)
    majority_labels.append(majority_label)

In [ ]:
# 2. 데이터 경로 설정 및 최적화
# Google Drive에서 로컬 디스크로 데이터 복사 (한 번만 실행)
local_data_path = '/content/data/'
if not os.path.exists(local_data_path):
    !cp -r '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data/' '/content/data/'

data_path = local_data_path  # 로컬 디스크 경로로 변경

In [ ]:
# 3. 모델 로드
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # 예시 모델 이름
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# GPU 사용 설정 (가능한 경우)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()  # 평가 모드로 설정

In [ ]:
# 4. 데이터셋 및 데이터로더 구현

class TextDataset(Dataset):
    def __init__(self, text_files):
        self.text_files = text_files

    def __len__(self):
        return len(self.text_files)

    def __getitem__(self, idx):
        text_file = self.text_files[idx]
        with open(text_file, 'r', encoding='utf-8') as file:
            raw_text = file.read().strip()
        return raw_text

# 유효한 텍스트 파일 리스트 생성
text_files = []
valid_sample_ids = []
for sample_id in sample_ids:
    text_filename = f"{sample_id}.txt"
    text_file = os.path.join(data_path, text_filename)
    if os.path.exists(text_file):
        text_files.append(text_file)
        valid_sample_ids.append(sample_id)
    else:
        print(f"{text_filename} 파일을 찾을 수 없습니다.")

# 데이터셋 및 데이터로더 생성
text_dataset = TextDataset(text_files)
batch_size = 32  # 배치 크기 설정
text_dataloader = DataLoader(text_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# 5. 예측 수행
predicted_labels = []
all_majority_labels = []
total_batches = len(text_dataloader)
print(f"총 배치 수: {total_batches}")

for batch_idx, batch_texts in enumerate(text_dataloader):
    # 진행 상황 출력
    if batch_idx % 10 == 0:
        print(f"{batch_idx}/{total_batches} 배치 처리 중...")

    # 텍스트 토크나이즈
    inputs = tokenizer(
        list(batch_texts),
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=128
    )
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # 모델 예측
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predicted_labels.extend(preds)


In [ ]:
# 6. 라벨 정렬
# valid_sample_ids에 대응하는 majority_labels 추출
valid_indices = [sample_ids.index(sid) for sid in valid_sample_ids]
filtered_majority_labels = [majority_labels[i] for i in valid_indices]

# 7. 예측과 라벨 비교
# 정확도 계산
correct = sum([1 for pred, true in zip(predicted_labels, filtered_majority_labels) if pred == true])
accuracy = correct / len(predicted_labels)
print(f"모델의 정확도: {accuracy * 100:.2f}%")

# 분류 보고서 출력
print(classification_report(filtered_majority_labels, predicted_labels, target_names=['Negative', 'Neutral', 'Positive']))
